In [1]:

import os
import json
import functools
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
# Step 1: Read the config.json file
with open("D:\\Healthscore Codes\\containment_metric.json", 'r') as file:
    config_data = json.load(file)
    print(config_data)

{'features': [{'feature_name': 'SMART HELP', 'feature_id': '15', 'metrics': [{'Primary_intent': 'Equipment Support', 'Primary_intent_detail': 'SmartHelp', 'cont_display_metric_name': 'Smarthelp_containment_rate', 'cont_metric_seq_num': '1'}, {'Primary_intent': 'Equipment Support', 'Primary_intent_detail': 'PnP', 'cont_display_metric_name': 'PnP_containment_rate', 'cont_metric_seq_num': '2'}]}]}


In [3]:
def create_dataframe_from_json(data):
    # Flatten the JSON and create a list of dictionaries
    flattened_data = []

    for feature in data["features"]:
        feature_name = feature["feature_name"]
        feature_id = feature["feature_id"]

        for metric in feature["metrics"]:
            flattened_data.append({
                "hs_feature_name": feature_name,
                "hs_feature_id": feature_id,
                "primary_intent": metric["Primary_intent"],
                "primary_intent_detail": metric["Primary_intent_detail"],
                "cont_display_metric_name": metric["cont_display_metric_name"],
                "cont_metric_seq_num": metric["cont_metric_seq_num"]
            })

    # Create the DataFrame
    df = pd.DataFrame(flattened_data)

    # Add the 'Containment_metric_id' column as a sequence number
    df["containment_metric_id"] = range(1, len(df) + 1)
    df['create_dt'] = pd.to_datetime('today').normalize().date()

    # Reorder the columns if needed
    df = df[["containment_metric_id", "hs_feature_name", "hs_feature_id", "primary_intent", "primary_intent_detail", "cont_display_metric_name", "cont_metric_seq_num", "create_dt"]]

    return df

In [4]:
# Call the function with JSON data
df = create_dataframe_from_json(config_data)

# Print the DataFrame
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 500)
print(df.head())

   containment_metric_id hs_feature_name hs_feature_id     primary_intent  \
0                      1      SMART HELP            15  Equipment Support   
1                      2      SMART HELP            15  Equipment Support   

  primary_intent_detail    cont_display_metric_name cont_metric_seq_num  \
0             SmartHelp  Smarthelp_containment_rate                   1   
1                   PnP        PnP_containment_rate                   2   

    create_dt  
0  2025-02-17  
1  2025-02-17  


In [5]:
# Get unique values from the columns
primary_intent_values = df['primary_intent'].unique()
primary_intent_detail_values = df['primary_intent_detail'].unique()

In [6]:
print(primary_intent_values)
print(primary_intent_detail_values)

['Equipment Support']
['SmartHelp' 'PnP']


In [7]:
# Convert arrays to strings formatted for SQL IN clauses
primary_intent_str = "', '".join(primary_intent_values)
primary_intent_detail_str = "', '".join(primary_intent_detail_values)

In [8]:
print(primary_intent_str)
print(primary_intent_detail_str)

Equipment Support
SmartHelp', 'PnP


In [9]:
# Build the dynamic query
second_table_query = f"""
SELECT 		
    primary_intent_detail,  		
    CAST(contact_dt AS DATE) AS contact_dt, 		
    COUNT(DISTINCT sub_contact_id) AS sub_contact_id, 		
    COUNT(DISTINCT CASE WHEN selfservice_containment = 1 THEN sub_contact_id END) AS contained, 		
    CASE 		
        WHEN COUNT(DISTINCT sub_contact_id) > 0 THEN		
            ROUND(CAST(SUM(CASE WHEN selfservice_containment = 1 THEN 1 ELSE 0 END) AS DOUBLE) 		
            / COUNT(DISTINCT sub_contact_id) * 100, 2)		
        ELSE		
0		
    END AS containment_rate		
FROM 		
    ota_data_assets_temp.omni_intent_cntct_fact 		
WHERE 		
    CAST(contact_dt AS DATE) BETWEEN date_add('day', -90, DATE '2024-08-27') AND DATE '2024-08-27' 		
    AND primary_intent IN ('{primary_intent_str}')
    AND initial_channel = 'CoxApp'		
    AND lob = 'R'		
    AND primary_intent_detail IN ('{primary_intent_detail_str}')
GROUP BY 		
    primary_intent_detail, contact_dt		
ORDER BY 		
    contact_dt DESC
"""


In [10]:
print(second_table_query)


SELECT 		
    primary_intent_detail,  		
    CAST(contact_dt AS DATE) AS contact_dt, 		
    COUNT(DISTINCT sub_contact_id) AS sub_contact_id, 		
    COUNT(DISTINCT CASE WHEN selfservice_containment = 1 THEN sub_contact_id END) AS contained, 		
    CASE 		
        WHEN COUNT(DISTINCT sub_contact_id) > 0 THEN		
            ROUND(CAST(SUM(CASE WHEN selfservice_containment = 1 THEN 1 ELSE 0 END) AS DOUBLE) 		
            / COUNT(DISTINCT sub_contact_id) * 100, 2)		
        ELSE		
0		
    END AS containment_rate		
FROM 		
    ota_data_assets_temp.omni_intent_cntct_fact 		
WHERE 		
    CAST(contact_dt AS DATE) BETWEEN date_add('day', -90, DATE '2024-08-27') AND DATE '2024-08-27' 		
    AND primary_intent IN ('Equipment Support')
    AND initial_channel = 'CoxApp'		
    AND lob = 'R'		
    AND primary_intent_detail IN ('SmartHelp', 'PnP')
GROUP BY 		
    primary_intent_detail, contact_dt		
ORDER BY 		
    contact_dt DESC



In [11]:
second_table_df = pd.read_csv(r"D:\Healthsore Data\hs_new_metric.csv")

In [12]:
second_table_df.head()

,primary_intent_detail,contact_dt,sub_contact_id,contained,containment_rate
0,PnP,2024-08-27,1839,1593,86.62
1,SmartHelp,2024-08-27,1453,1346,92.64
2,SmartHelp,2024-08-26,1500,1399,93.27
3,PnP,2024-08-26,1932,1658,85.82
4,PnP,2024-08-25,1900,1680,88.42


In [13]:
# Merge the two dataframes on 'primary_intent_detail' using a left join
combined_df = pd.merge(second_table_df, df, on='primary_intent_detail', how='left')


In [14]:
combined_df.head()

,primary_intent_detail,contact_dt,sub_contact_id,contained,containment_rate,containment_metric_id,hs_feature_name,hs_feature_id,primary_intent,cont_display_metric_name,cont_metric_seq_num,create_dt
0,PnP,2024-08-27,1839,1593,86.62,2,SMART HELP,15,Equipment Support,PnP_containment_rate,2,2025-02-17
1,SmartHelp,2024-08-27,1453,1346,92.64,1,SMART HELP,15,Equipment Support,Smarthelp_containment_rate,1,2025-02-17
2,SmartHelp,2024-08-26,1500,1399,93.27,1,SMART HELP,15,Equipment Support,Smarthelp_containment_rate,1,2025-02-17
3,PnP,2024-08-26,1932,1658,85.82,2,SMART HELP,15,Equipment Support,PnP_containment_rate,2,2025-02-17
4,PnP,2024-08-25,1900,1680,88.42,2,SMART HELP,15,Equipment Support,PnP_containment_rate,2,2025-02-17


In [15]:
second_table_columns = ['containment_metric_id', 'primary_intent', 'primary_intent_detail', 'sub_contact_id', 'contained', 'contact_dt', 'containment_rate', 'hs_feature_name', 'cont_display_metric_name']
second_tabledf = combined_df[second_table_columns]
second_tabledf.head()

,containment_metric_id,primary_intent,primary_intent_detail,sub_contact_id,contained,contact_dt,containment_rate,hs_feature_name,cont_display_metric_name
0,2,Equipment Support,PnP,1839,1593,2024-08-27,86.62,SMART HELP,PnP_containment_rate
1,1,Equipment Support,SmartHelp,1453,1346,2024-08-27,92.64,SMART HELP,Smarthelp_containment_rate
2,1,Equipment Support,SmartHelp,1500,1399,2024-08-26,93.27,SMART HELP,Smarthelp_containment_rate
3,2,Equipment Support,PnP,1932,1658,2024-08-26,85.82,SMART HELP,PnP_containment_rate
4,2,Equipment Support,PnP,1900,1680,2024-08-25,88.42,SMART HELP,PnP_containment_rate


In [34]:
# Convert 'contact_dt' to datetime
second_table_df['contact_dt'] = pd.to_datetime(second_table_df['contact_dt'])

# Pivot the table to reshape
pivot_df = second_table_df.pivot_table(
    index='primary_intent_detail', 
    columns='contact_dt', 
    values=['sub_contact_id', 'contained', 'containment_rate'],
    aggfunc='first'
)

# Flatten the multi-index columns and format the date
pivot_df.columns = [f'{col[1].strftime("%d-%m-%Y")}_{col[0]}' for col in pivot_df.columns]

# Reset index
pivot_df = pivot_df.reset_index()

In [35]:
# Display the transformed DataFrame
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 500)
print(pivot_df)

  primary_intent_detail  29-05-2024_contained  30-05-2024_contained  \
0                   PnP                  1446                  1514   
1             SmartHelp                  1463                  1577   

   31-05-2024_contained  01-06-2024_contained  02-06-2024_contained  \
0                  1460                  1425                  1321   
1                  1564                  1522                  1268   

   03-06-2024_contained  04-06-2024_contained  05-06-2024_contained  \
0                  1393                  1501                  1480   
1                  1553                  1602                  1565   

   06-06-2024_contained  07-06-2024_contained  08-06-2024_contained  \
0                  1565                  1601                  1316   
1                  1496                  1649                  1355   

   09-06-2024_contained  10-06-2024_contained  11-06-2024_contained  \
0                  1349                  1466                  1576   
1 

In [36]:
len_cols = (len(pivot_df.columns))
print(len_cols)

274


In [38]:
result_df = pd.merge(df, pivot_df, on='primary_intent_detail')

In [39]:
result_df.head()

,containment_metric_id,hs_feature_name,hs_feature_id,primary_intent,primary_intent_detail,cont_display_metric_name,cont_metric_seq_num,create_dt,29-05-2024_contained,30-05-2024_contained,31-05-2024_contained,01-06-2024_contained,02-06-2024_contained,03-06-2024_contained,04-06-2024_contained,05-06-2024_contained,06-06-2024_contained,07-06-2024_contained,08-06-2024_contained,09-06-2024_contained,10-06-2024_contained,11-06-2024_contained,12-06-2024_contained,13-06-2024_contained,14-06-2024_contained,15-06-2024_contained,16-06-2024_contained,17-06-2024_contained,18-06-2024_contained,19-06-2024_contained,20-06-2024_contained,21-06-2024_contained,22-06-2024_contained,23-06-2024_contained,24-06-2024_contained,25-06-2024_contained,26-06-2024_contained,27-06-2024_contained,28-06-2024_contained,29-06-2024_contained,30-06-2024_contained,01-07-2024_contained,02-07-2024_contained,03-07-2024_contained,04-07-2024_contained,05-07-2024_contained,06-07-2024_contained,07-07-2024_contained,08-07-2024_contained,09-07-2024_contained,10-07-2024_contained,11-07-2024_contained,12-07-2024_contained,13-07-2024_contained,14-07-2024_contained,15-07-2024_contained,16-07-2024_contained,17-07-2024_contained,18-07-2024_contained,19-07-2024_contained,20-07-2024_contained,21-07-2024_contained,22-07-2024_contained,23-07-2024_contained,24-07-2024_contained,25-07-2024_contained,26-07-2024_contained,27-07-2024_contained,28-07-2024_contained,29-07-2024_contained,30-07-2024_contained,31-07-2024_contained,01-08-2024_contained,02-08-2024_contained,03-08-2024_contained,04-08-2024_contained,05-08-2024_contained,06-08-2024_contained,07-08-2024_contained,08-08-2024_contained,09-08-2024_contained,10-08-2024_contained,11-08-2024_contained,12-08-2024_contained,13-08-2024_contained,14-08-2024_contained,15-08-2024_contained,16-08-2024_contained,17-08-2024_contained,18-08-2024_contained,19-08-2024_contained,20-08-2024_contained,21-08-2024_contained,22-08-2024_contained,23-08-2024_contained,24-08-2024_contained,25-08-2024_contained,26-08-2024_contained,27-08-2024_contained,29-05-2024_containment_rate,30-05-2024_containment_rate,31-05-2024_containment_rate,01-06-2024_containment_rate,02-06-2024_containment_rate,03-06-2024_containment_rate,04-06-2024_containment_rate,05-06-2024_containment_rate,06-06-2024_containment_rate,07-06-2024_containment_rate,08-06-2024_containment_rate,09-06-2024_containment_rate,10-06-2024_containment_rate,11-06-2024_containment_rate,12-06-2024_containment_rate,13-06-2024_containment_rate,14-06-2024_containment_rate,15-06-2024_containment_rate,16-06-2024_containment_rate,17-06-2024_containment_rate,18-06-2024_containment_rate,19-06-2024_containment_rate,20-06-2024_containment_rate,21-06-2024_containment_rate,22-06-2024_containment_rate,23-06-2024_containment_rate,24-06-2024_containment_rate,25-06-2024_containment_rate,26-06-2024_containment_rate,27-06-2024_containment_rate,28-06-2024_containment_rate,29-06-2024_containment_rate,30-06-2024_containment_rate,01-07-2024_containment_rate,02-07-2024_containment_rate,03-07-2024_containment_rate,04-07-2024_containment_rate,05-07-2024_containment_rate,06-07-2024_containment_rate,07-07-2024_containment_rate,08-07-2024_containment_rate,09-07-2024_containment_rate,10-07-2024_containment_rate,11-07-2024_containment_rate,12-07-2024_containment_rate,13-07-2024_containment_rate,14-07-2024_containment_rate,15-07-2024_containment_rate,16-07-2024_containment_rate,17-07-2024_containment_rate,18-07-2024_containment_rate,19-07-2024_containment_rate,20-07-2024_containment_rate,21-07-2024_containment_rate,22-07-2024_containment_rate,23-07-2024_containment_rate,24-07-2024_containment_rate,25-07-2024_containment_rate,26-07-2024_containment_rate,27-07-2024_containment_rate,28-07-2024_containment_rate,29-07-2024_containment_rate,30-07-2024_containment_rate,31-07-2024_containment_rate,01-08-2024_containment_rate,02-08-2024_containment_rate,03-08-2024_containment_rate,04-08-2024_containment_rate,05-08-2024_containment_rate,06-08-2024_c